Make sure you fill your name and NetID below:

In [ ]:
NAME = "Sean Kim"
NET_ID = "skk135"

---

---
# Assignment 3: RNN, LSTM, Attention, and Transformers

In this assignment, you will implement neural machine translation (NMT) models using:

1. RNNs
2. LSTMs and LSTMs with attention
3. Transformers.

As in the previous assignments, you will see code blocks that look like this:
```python
###############################################################################
# TODO: Create a variable x with value 3.7
###############################################################################
pass
# END OF YOUR CODE
```

You should replace the `pass` statement with your own code and leave the blocks intact, like this:
```python
###############################################################################
# TODO: Create a variable x with value 3.7
###############################################################################
x = 3.7
# END OF YOUR CODE
```

Also, please remember:
- Do not write or modify any code outside of code blocks
- Do not add or delete any cells from the notebook. You may add new cells to perform scatch work, but delete them before submitting.
- Run all cells before submitting. You will only get credit for code that has been run.


## Setup

First let's import some libraries that will be useful in this assignment.


In [ ]:
import zipfile
import matplotlib.pyplot as plt
import random
import collections
import numpy as np
import zipfile
import torch
import os
import torch.nn as nn
import torch.nn.functional as F

def seed(seed):
  torch.manual_seed(seed)
  np.random.seed(seed)

Make sure you are using the GPU.

In [ ]:
if torch.cuda.is_available():
  print('Good to go!')
  device = torch.device('cuda:0')
else:
  print('Using CPU.')
  device = torch.device('cpu')

For this assignment, we will use an English-to-French dataset. As shown below, the dataset contains multiple lines each of which has an English sentence and its French translation separated by a tab. In this problem, since English is translated to French, English is the source language and French is the target language. Note that each text sequence is of variable lengnth and can be just one sentence or a paragraph of multiple sentences.

In [ ]:
def download_if_not_exist(file_name):
  
  if not os.path.exists(file_name):
    import urllib.request
    DATA_URL = 'https://download.pytorch.org/tutorial/data.zip'

    file_name, _ = urllib.request.urlretrieve(DATA_URL, './data.zip')
    
  return file_name

def read_raw(file_name):
  file_name = download_if_not_exist(file_name)
  
  with zipfile.ZipFile(file_name, 'r') as fzip:
    raw_text = fzip.read(file_name.split('.')[-2][1:] + '/eng-fra.txt').decode('utf-8')
  return raw_text

In [ ]:
raw_text = read_raw('./data.zip')
print(raw_text[:200])

Next we'll do some preprocessing on this raw text. We need to replace special symbols (non-breaking spaces) with spaces, convert all characters to lower case, and insert a space between words and punctuation marks.

In [ ]:
def preprocess_raw(text):
  text = text.replace('\u202f', ' ').replace('\xa0', ' ')
  out = ''
  for i, char in enumerate(text.lower()):
    if char in (',', '!', '.') and i > 0 and text[i-1] != ' ':
      out += ' '
    out += char
  return out

We further split the source-target pairs into a source list and a target list. We use word-level tokenization here. 

In [ ]:
def split_source_target(text, max_len):
  source, target = [], []
  for i, line in enumerate(text.split('\n')):
    if i > 5000: # we only use 5000 pairs of translation
      break
    parts = line.split('\t')
    if len(parts) == 2:
      src_tokens = parts[0].split(' ')
      tgt_tokens = parts[1].split(' ')
      if (len(src_tokens) <= max_len) and (len(tgt_tokens) <= max_len):
        source.append(src_tokens)
        target.append(tgt_tokens)
  return source, target

In [ ]:
def prepare_data(raw_text, max_len=10000):
  text = preprocess_raw(raw_text)
  source, target = split_source_target(text, max_len)
  return source, target

source, target = prepare_data(raw_text)

Using the whole dataset takes too much memory, and it is hard to train with a large vocabulary. Thus, we will filter out some words by looking at the statistical properties of the dataset.

In [ ]:
def len_dis(text):
  lens = [len(line) for line in text]
  len_counter = collections.Counter(lens)

  lens = np.array(list(len_counter.keys()))
  sort_idx = np.argsort(lens)
  lens_sort = lens[sort_idx]
  len_counts = np.array(list(len_counter.values()))
  len_counts_sort = len_counts[sort_idx]
  p = np.cumsum(len_counts_sort) / len_counts_sort.sum()
  return p, lens_sort
  
src_p, src_lens_sort = len_dis(source)
tgt_p, tgt_lens_sort = len_dis(target)
plt.plot(src_lens_sort, src_p, 'r-', label='eng')
plt.plot(tgt_lens_sort, tgt_p, 'g-', label='fra')
plt.title('Cumulative Distribution of Sentence Length')
plt.legend()
plt.show()

From the above plots, we can see that more than 90% of the sentences have a length of less than 8. Thus, we can filter out sentences of length greater than 8. We also filter out words that occur less than 5 times in the corpus.

In [ ]:
# hyper-param
MAX_LEN = 8
MIN_FREQ = 5

### Build Vocabulary

Each word needs a unique index, and the words that have been filtered out need a special token to represent them. The following class Vocab is used to build the vocabulary. Some basic helper functions or dictionaries are also provided:
- Dictionary word2index: Convert word string into index: 
- Dictionary index2word: Convert index into word string
- helper function _build_vocab(): Build dictionaries for converting from words to indices and vice versa
- Word Counter, num_word: Record the total number of unique tokens in the vocabulary 
    
There are 4 special tokens added in the vocabulary:
- 'pad': padding token. Sentences shorter than MAX_LEN is padded by this symbol to make the length to MAX_LEN
- 'bos': beginning of sentence. This indicates the beginning of a sentence
- 'eos': end of sentence. This indicates the end of a sentence
- 'unk': unknown word. This represents words that have been filtered out (words that are not in the vocabulary)

In [ ]:
class Vocab():
  def __init__(self, name, tokens, min_freq):
    self.name = name
    self.index2word = {
      0: 'pad',
      1: 'bos',
      2: 'eos',
      3: 'unk'
    }
    self.word2index = {v: k for k, v in self.index2word.items()}
    self.num_word = 4
    token_freq = collections.Counter(tokens)
    tokens = [token for token in tokens if token_freq[token] >= MIN_FREQ]
    self._build_vocab(tokens)
    
  def _build_vocab(self, tokens):
    for token in tokens:
      if token not in self.word2index:
        self.word2index[token] = self.num_word
        self.index2word[self.num_word] = token
        self.num_word += 1
        
  def __getitem__(self, tokens):
    if not isinstance(tokens, (list, tuple)):
      return self.word2index.get(tokens, self.word2index['unk'])
    else:
      return [self.__getitem__(token) for token in tokens]

### Build Dataset

The dataset pipeline involves the following steps:
- For target language, every sentence will be 'sandwiched' with the 'bos' token and the 'eos' token.
- Every sentence that has a length less than MAX_LEN will be padded to the MAX_LEN with the *padding_token*.
- The dataset should return the converted tensor and the corresponding valid length before padding.
- We use the Pytorch *DataLoader* API to build the dataset generator.

For the purposes of this assignment, we will train and evaluate on only the training data. This isn't ideal because we do not know if we are  overfitting to the training data, but it is fine for instructional purposes. In practice (eg. for your projects), you should make sure to split your data into training/validation/test datasets.

In [ ]:
def build_vocab(name, tokens, min_freq):
  tokens = [token for line in tokens for token in line]
  return Vocab(name, tokens, min_freq)

def build_vocabs(lang_src, lang_tgt, src_text, tgt_text):
  vocab_src = build_vocab(lang_src, src_text, MIN_FREQ)
  vocab_tgt = build_vocab(lang_tgt, tgt_text, MIN_FREQ)
  return vocab_src, vocab_tgt

def pad(line, padding_token):
  return line + [padding_token] * (MAX_LEN + 2 - len(line))

def build_tensor(text, lang, is_source):
  lines = [lang[line] for line in text]
  if not is_source:
    lines = [[lang['bos']] + line + [lang['eos']] for line in lines]
  array = torch.tensor([pad(line, lang['pad']) for line in lines])
  valid_len = (array != lang['pad']).sum(1)
  return array, valid_len

def load_data_nmt(batch_size=2):
  lang_eng, lang_fra = build_vocabs('eng', 'fra', source, target)
  src_array, src_valid_len = build_tensor(source, lang_eng, True)
  tgt_array, tgt_valid_len = build_tensor(target, lang_fra, False)
  train_data = torch.utils.data.TensorDataset(
    src_array, src_valid_len, tgt_array, tgt_valid_len)
  print(train_data[0])
  train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
  return lang_eng, lang_fra, train_iter


source, target = prepare_data(raw_text, max_len=MAX_LEN)
vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size=2)
print('Vocabulary size of source language: {}'.format(vocab_eng.num_word))
print('Vocabulary size of target language: {}'.format(vocab_fra.num_word))
print('Total number of sentence pairs: {}'.format(len(source)))

## Sequence to Sequence with RNN (baseline)

In this section, we provide the implementation of the seq2seq RNN baseline model. You do not need to implement any code in this section, but you should read and understand what the code is doing because you will need to implement something similar in subsequent sections. The following figure highlights the architecture of the seq2seq model. An encoder RNN encodes the input sequence into its hidden state, and passes the last hidden state to the decoder RNN. The decoder generates the target sequence.

Implementation Details:

- Embedding: We have represented each word with an integer or one-hot vector. We need an embedding layer to map an input word to its embedding vector.
- Encoder: A vanilla RNN is used to encode a source sequence. The final hidden state is returned as output and passed to the decoder RNN.
- Decoder: Another vanilla RNN is implemented to generate the target sequence. The hidden state is initialized with the last hidden state from the encoder.
- Encoder-Decoder: The class NMTRNN is built by combining the encoder and the decoder, and yields the loss and predictions.
- Loss: We have padded all sentences so that they have the same MAX_LEN. Thus, when we compute the loss, the loss from those padding_tokens should be masked out.

<div>
<img src="https://raw.githubusercontent.com/dsgiitr/d2l-pytorch/24e89824c154c2afc419c5dadec9622e490b99bb/img/seq2seq.svg" width="600"/>
</div>
Image source: https://github.com/dsgiitr/d2l-pytorch/blob/master/img/seq2seq.svg

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size):
    super(Encoder, self).__init__()
    """
    inputs:
      vocab_size: int, the number of words in the vocabulary
      embedding_dim: int, dimension of the word embedding
      hidden_size: int, dimension of the hidden state of vanilla RNN
    """
    self.embedding = nn.Embedding(vocab_size, embedding_dim) # embedding layer
    self.enc = nn.RNN(embedding_dim, hidden_size, batch_first=True)
    self.hidden_size = hidden_size
    
  def forward(self, sources, valid_len):
    """
    Inputs:
      source: tensor of size (N, T), where N is the batch size, T is the length of the sequence(s)
      valid_len: tensor of size (N,), indicating the valid length of sequence(s) (the length before padding)
    """
    word_embedded = self.embedding(sources)
    N = word_embedded.shape[0]
    
    h = sources.new_zeros(1, N, self.hidden_size).float() # initialize hidden state with zeros
    
    o, h = self.enc(word_embedded, h)
    
    return o[np.arange(N), valid_len] # return the hidden state of the valid last time step

class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size):
    super(Decoder, self).__init__()
    """
    inputs:
      vocab_size: int, the number of words in the vocabulary
      embedding_dim: int, dimension of the word embedding
      hidden_size: int, dimension of the hidden state of vanilla RNN
    """
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.enc = nn.RNN(embedding_dim, hidden_size, batch_first=True)
    self.output_emb = nn.Linear(hidden_size, vocab_size)
    self.hidden_size = hidden_size
    
  def forward(self, h, target):
    word_embedded = self.embedding(target)
    N, T = word_embedded.shape[:2]
    
    o, h = self.enc(word_embedded, h.view(1,N,self.hidden_size))
    pred = self.output_emb(o)
    return pred, h

class NMTRNN(nn.Module):
  def __init__(self, src_vocab_size, tgt_vocab_size, embedding_dim, hidden_size):
    super(NMTRNN, self).__init__()
    self.enc = Encoder(src_vocab_size, embedding_dim, hidden_size)
    self.dec = Decoder(tgt_vocab_size, embedding_dim, hidden_size)
    
  def forward(self, src, src_len, tgt, tgt_len):
    h = self.enc(src, src_len)
    T = tgt.shape[1]
    
    pred, _ = self.dec(h, tgt)
    loss = 0
    for t in range(T-1):
      # target sequence should shift by one time-step, because we are predicting the next word
      # Note that in principle the `ignore_index` parameter can be set to 0, which is for the `pad` token. 
      # Due to version change in PyTorch, this may have a bug. Therefore we do not use it here. 
      loss = loss + F.nll_loss(F.log_softmax(pred[:, t]), tgt[:, t+1]) #, ignore_index=0)

    return loss, pred.argmax(dim=-1)

  def predict(self, src, src_len, tgt, tgt_len):
      """
      When predicting a sequence given the 'bos' token, the input for the next step is the predicted
      token from the previous time step.
      """
      h = self.enc(src, src_len)

      inputs = tgt[:, :1]
      preds = []
      for t in range(MAX_LEN+1): # plus the 'eos' token
        pred, h = self.dec(h, inputs)
        preds.append(pred)
        inputs = pred.argmax(dim=-1)
        
      pred = torch.cat(preds, dim=1).argmax(dim=-1)
      return pred
        

In [ ]:
def train_rnn(net, train_iter, lr, epochs, device):
  # training
  net = net.to(device)

  optimizer = torch.optim.Adam(net.parameters(), lr=lr)
  loss_list = []
  print_interval = len(train_iter)
  total_iter = epochs * len(train_iter)
  for e in range(epochs):
    net.train()
    for i, train_data in enumerate(train_iter):
      train_data = [ds.to(device) for ds in train_data]

      loss, pred = net(*train_data)

      loss_list.append(loss.mean().detach())
      optimizer.zero_grad()
      loss.mean().backward()
      optimizer.step()

      step = i + e * len(train_iter)
      if step % print_interval == 0:
        print('iter {} / {}\tLoss:\t{:.6f}'.format(step, total_iter, loss.mean().detach()))
        print('pred:\t {}\n'.format(pred[0].detach().cpu()))
        print('tgt:\t {}\n'.format(train_data[2][0][1:].cpu()))
  return loss_list

seed(1)
batch_size = 32
lr = 1e-3
epochs = 50

embedding_dim = 250
hidden_size = 128

vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size)
rnn_net = NMTRNN(vocab_eng.num_word, vocab_fra.num_word, embedding_dim, hidden_size)

rnn_loss_list = train_rnn(rnn_net, train_iter, lr, epochs, device)

### RNN Loss Curve

Plot the loss curve over time.

In [ ]:
# save the loss curve figure in a file for the report
if device != "cpu":
    rnn_loss_list_cpu = [ele.cpu() for ele in rnn_loss_list]
else:
    rnn_loss_list_cpu = run_loss_list
plt.plot(np.arange(len(rnn_loss_list_cpu)), rnn_loss_list_cpu)
plt.title('Loss Curve of Baseline')


### Prediction Accuracy

Print out 5 prediction samples, and calculate the prediction accuracy over the training dataset. You will see an accuracy of over 80%.

In [ ]:
def comp_acc(pred, gt, valid_len):
  N, T_gt = gt.shape[:2]
  _, T_pr = pred.shape[:2]
  assert T_gt == T_pr, 'Prediction and target should have the same length.'
  len_mask = torch.arange(T_gt).expand(N, T_gt)
  len_mask = len_mask < valid_len[:, None]
  
  pred_crr = (pred == gt).float() * len_mask.float() # filter out the 'bos' token
  pred_acc = pred_crr.sum(dim=-1) / (valid_len - 1).float() # minus the 'bos' token
  return pred_acc
  
def evaluate_rnn(net, train_iter, device):
  acc_list = []
  for i, train_data in enumerate(train_iter):
    train_data = [ds.to(device) for ds in train_data]

    pred = net.predict(*train_data)

    pred_acc = comp_acc(pred.detach().cpu(), train_data[2].detach().cpu()[:, 1:], train_data[3].cpu())
    acc_list.append(pred_acc)
    if i < 5:# print 5 samples from 5 batches
      pred = pred[0].detach().cpu()
      pred_seq = []
      for t in range(MAX_LEN+1):
        pred_wd = vocab_fra.index2word[pred[t].item()] 
        if pred_wd != 'eos':
          pred_seq.append(pred_wd)

      print('pred:\t {}\n'.format(pred_seq))
      print('tgt:\t {}\n'.format([vocab_fra.index2word[t.item()] for t in train_data[2][0][1:].cpu()]))

  print('Prediction Acc.: {:.4f}'.format(torch.cat(acc_list).mean()))
  
seed(1)
batch_size = 32

vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size)

evaluate_rnn(rnn_net, train_iter, device)

## Sequence to Sequence with LSTM and Attention

Now let's try to improve our model by using an LSTM and the attention mechanism.


### LSTM

LSTMs eliminate the gradient explosion/vanishing problem. Its state and gate update at each time step can be summarized as follows:

$$
\begin{align*}
&\text{State Update} &&& C_t &= F_t \odot C_{t-1} + I_t \odot \tilde{C}_t \\
&\text{Hidden States} &&& H_t &= O_t \odot \text{tanh}(C_t) \\
&\text{Proposal} &&& \tilde{C}_t &= \text{tanh}( X_tW_{xc} + H_{t-1}W_{hc} + b_c ) \\
&\text{Input Gate} &&& I_t &= \sigma( X_tW_{xi} + H_{t-1}W_{hi} + b_i ) \\
&\text{Forget Gate} &&& F_t &= \sigma( X_tW_{xf} + H_{t-1}W_{hf} + b_f ) \\
&\text{Output Gate} &&& O_t &= \sigma( X_tW_{xo} + H_{t-1}W_{ho} + b_o ) \\
\end{align*}
$$

Implement the LSTM class below. In particular,
-  Complete the initialization function *init_params()*. Weights should be initialized using `torch.randn` multiplied with a scale of 0.1. Biases should be initialized to 0.
- Complete the function *lstm()* which performs the feed-forward pass of LSTM. **Do not** use `nn.LSTM` or `nn.LSTMCell` in your implementation.

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, device):
    super(LSTM, self).__init__()
    self.device = device
    self.params = nn.ParameterList(self.init_params(input_size, hidden_size))
    """
    Inputs:
      input_size: int, feature dimension of input sequence
      hidden_size: int, feature dimension of hidden state
      device: torch.device()
    """
  
  def init_params(self, input_size, hidden_size):
    """
    Inputs:
      input_size: int, feature dimension of input sequence
      hidden_size: int, feature dimension of hidden state
      
    Outputs:
      Weights for proposal: W_xc, W_hc, b_c
      Weights for input gate: W_xi, W_hi, b_i
      Weights for forget gate: W_xf, W_hf, b_f
      Weights for output gate: W_xo, W_ho, b_o
    """
    W_xc, W_hc, b_c = None, None, None
    W_xi, W_hi, b_i = None, None, None
    W_xf, W_hf, b_f = None, None, None
    W_xo, W_ho, b_o = None, None, None
    ##############################################################################
    # TODO: Initialize the weights and biases. The result will be stored in 
    # `params` below. Weights should be initialized using `torch.randn` multiplied 
    # with the scale (0.1). Biases should be initialized to 0.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    
    params = [W_xc, W_hc, b_c, W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o]
    return params

  
  def lstm(self, X, state):
    """
    Inputs:
      X: tuple of tensors (src, src_len). src, size (N, D_in) or (N, T, D_in), where N is the batch size,
        T is the length of the sequence(s). src_len, size of (N,), is the valid length for each sequence.
        
      state: tuple of tensors (h, c). h, size of (N, hidden_size) is the hidden state of LSTM. c, size of 
            (N, hidden_size), is the memory cell of the LSTM.
      
    Outputs:
      o: tensor of size (N, T, hidden_size). Contains the output features (the hidden state H_t) for each t.
      state: the same as input state. Contains the hidden state H_T and cell state C_T for the last timestep T.
    """
    
    src, src_len = X
    h, c = state

    # make sure always has a T dim
    if len(src.shape) == 2:
      src = src.unsqueeze(1)

    N, T, D_in = src.shape
    W_xc, W_hc, b_c, W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o = self.params
    o = []
    ##############################################################################
    # TODO: Implement the forward pass of the LSTM.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    state = (h, c)
    return o, state
  
  def forward(self, inputs, state):
    return self.lstm(inputs, state)

Check that your output has the correct shape. You should see:

```
torch.Size([12, 8, 5])
torch.Size([12, 5])
torch.Size([12, 5])
```

In [ ]:
test_lstm = LSTM(10, 5, torch.device('cpu'))
test_src = torch.ones(12, 8, 10)
test_src_len = torch.ones(12) * 8
test_h = torch.zeros(12, 5).float()
test_c = torch.zeros(12, 5).float()

test_o, test_state = test_lstm((test_src, test_src_len), (test_h, test_c))

print(test_o.shape)
print(test_state[0].shape)
print(test_state[1].shape)


### Attention Mechanism

Another improvement we can make to our model is the Attention Mechanism. An example illustrating why applying attention mechanisms can improve the performance is shown in the picture below. An English sentence and its Chinese is visualized and aligned into blue boxes and red boxes, respectively. It can be seen that the Chinese character '她' has a long distance from its English counterpart, 'she'. Since only the final hidden state is passed to the decoder, it's hard for the baseline model to 'attend' to information a long time ago.

<div>
<img src="https://lilianweng.github.io/lil-log/assets/images/encoder-decoder-example.png" width="600"/>
</div>
Image source: https://lilianweng.github.io/lil-log/assets/images/encoder-decoder-example.png

- **Attention**

    Given a query, $\mathbf{q} \in R^{d_q}$, and a set of $N$ (key, value) pairs, $\{ \mathbf{k}_i, \mathbf{v}_i\}^N$ where $k_i \in R^{d_k}$ and $v_i \in R^{d_v}$, the attention mechanism computes a weighted sum of values based on the normalized score obtained from the query and each key:
    $$
    \begin{align*}
    a_i &= \alpha(\mathbf{q}, \mathbf{k_i}) \\
    \mathbf{a} &= [a_1, ..., a_n] \\
    \mathbf{b} &= \text{softmax}(\mathbf{a}) \\
    \mathbf{o} &= \mathbf{b} \cdot \mathbf{V}\text{, where } \mathbf{V} = \{\mathbf{v}_i\}^N
    \end{align*}
    $$
    The $\alpha()$ function, which maps two vectors into a scalar, is the score function that can be chosen from a wide range of functions: e.g. the cosine function, dot-product function, scaled dot-product funtion and etc.


- **Masked Softmax**

For our machine translation task, the inputs and outputs may be of variable length (ie. each training example may have a different number of words). As shown above, we pad our inputs with a special `pad` token so that they all have the same length to make them easier to work with. However, when we take the softmax, we only want to include the non-`pad` items, so we need to write a special `masked_softmax` function to handle this. We can achieve the masking by setting masked elements to a large negative value. Then when we take the `exp`, those elements will be 0 and won't contribute to the softmax. We provide the implementation of this for you.

In [ ]:
def masked_softmax(X, valid_length):
  """
  inputs:
    X: 3-D tensor
    valid_length: 1-D or 2-D tensor
  """
  mask_value = -1e7 

  if len(X.shape) == 2:
    X = X.unsqueeze(1)

  N, n, m = X.shape

  if len(valid_length.shape) == 1:
    valid_length = valid_length.repeat_interleave(n, dim=0)
  else:
    valid_length = valid_length.reshape((-1,))

  mask = torch.arange(m)[None, :].to(X.device) >= valid_length[:, None]
  X.view(-1, m)[mask] = mask_value

  Y = torch.softmax(X, dim=-1)

  
  return Y

In [ ]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

- **Scaled Dot Product Attention**
    - The scaled dot-product attention uses the score function as: $\alpha(\mathbf{q}, \mathbf{k}) = \mathbf{q} \mathbf{k}^T / \sqrt{d}$, where $d$ is the dimension of query (which in this case is equal to the dimension of the keys). The following figures visualizes this process in matrix form, in which $Q \in \mathcal{R}^{m\times d_k}, \mathbf{K} \in \mathcal{R}^{n \times d_k}$, and $\mathbf{V} \in \mathcal{R}^{n \times d_v}$.

    <div>
    <img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" width="600"/>
    </div>
Image source: http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png

Implement the DotProductAttention below. Do not use any loops in your implementation.

In [ ]:
class DotProductAttention(nn.Module): 
  def __init__(self):
      super(DotProductAttention, self).__init__()

  def forward(self, query, key, value, valid_length=None):
    """
    inputs:
      query: tensor of size (B, n, d)
      key: tensor of size (B, m, d)
      value: tensor of size (B, m, dim_v)
      valid_length: (B, )

      B is the batch_size, n is the number of queries, m is the number of <key, value> pairs,
      d is the feature dimension of the query, and dim_v is the feature dimension of the value.

    Outputs:
      attention: tensor of size (B, n, dim_v), weighted sum of values
    """
    ##############################################################################
    # TODO: Implement the forward pass of DotProductAttention. Do not
    # use any loops in your implementation.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    return attention

### Correctness Check for DotProductAttention

Run the following snippet to check your implementation of DotProductAttention.

Expected output:

```
tensor([[[ 2.0000,  3.0000, 4.0000, 5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]])
```


In [ ]:
att = DotProductAttention()
keys = torch.ones((2,10,2),dtype=torch.float)
values = torch.arange((40), dtype=torch.float).view(1,10,4).repeat(2,1,1)
att(torch.ones((2,1,2),dtype=torch.float), keys, values, torch.FloatTensor([2, 6]))


- **MLP Attention**

    In MLP attention, we project both query and keys into $R^h$, add the results, and use a $\text{tanh}$ before multiplying by the values. The score function is defined as:
    
    $$
    \alpha(\mathbf{q}, \mathbf{k}) = \mathbf{v}^T\text{tanh}(W_k\mathbf{k} + W_q\mathbf{q})
    $$
    
    where $\mathbf{v}, \mathbf{W_k}\text{, and }\mathbf{W_q}$ are learnable parameters.
    
Implement the MLP attention in matrix form without using any loops.

In [ ]:
class MLPAttention(nn.Module):  
  def __init__(self, d_v, d_k, d_q):
    super(MLPAttention, self).__init__()
    """
    Inputs:
      d_k: feature dimension of key
      d_v: feature dimension of vector v
      d_q: feature dimension of query
    """
    ##############################################################################
    # TODO: Initialize learnable parameters
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

  def forward(self, query, key, value, valid_length):
    """
    inputs:
      query: tensor of size (B, n, d)
      key: tensor of size (B, m, d)
      value: tensor of size (B, m, dim_v)
      valid_length: either (B, )

      B is the batch_size, n is the number of queries, m is the number of <key, value> pairs,
      d is the feature dimension of the query, and dim_v is the feature dimension of the value.

    Outputs:
      attention: tensor of size (B, n, dim_v), weighted sum of values
    """
    ##############################################################################
    # TODO: Implement the forward pass of MLPAttention. Do not
    # use any loops in your implementation.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    return Y

### Correctness Check for MLPAttention

Run the following snippet to check your implementation of MLPAttention.

Expected output:

```
tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000],
         [ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000],
         [10.0000, 11.0000, 12.0000, 13.0000]]])
```

In [ ]:
atten = MLPAttention(4, 2, 2)
atten(torch.ones((2,2,2),dtype=torch.float), keys, values, torch.FloatTensor([2, 6]))

    
- **Using Attention in seq2seq Models**

    <div>
    <img src="https://d2l.ai/_images/seq2seq-attention.svg" width="600"/>
    </div>
Image source: https://d2l.ai/_images/seq2seq-attention.svg

    Now we want to add attention to the seq2seq model. As we previously stated, attention allows the decoder to have more direct access to previous states in the encoder. In the context of machine translation, when the decoder is predicting a word in the translation, it can focus on certain words in the original language. Therefore, we want the keys and the values of the attention layer to be the output of the encoder at each step. The query for the attention layer would be the decoder's previous hidden state. The output of the attention layer, referred to as the context, is concatenated with the decoder input and fed into the decoder.
    
    In rough pseudocode, this looks like:
    ```
    context = attention(query=h_prev, keys=encoder_output, values=encoder_output)
    decoder_input = concatenate([decoder_input, context])
    ```

### LSTM Encoder-Decoder


Build a seq2seq model with LSTM and attention.

- Complete the Encoder forward() function.
- Complete the Decoder forward() and predict() functions. The decoder should utilize the attention mechanism.
- Find a good learning rate for training this model. Feel free to add code here to test out different learning rates, but make sure that your best model is saved in `lstm_net`.

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, device):
    super(Encoder, self).__init__()
    """
    inputs:
      vocab_size: int, the number of words in the vocabulary
      embedding_dim: int, dimension of the word embedding
      hidden_size: int, dimension of vallina RNN
    """
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.enc = LSTM(embedding_dim, hidden_size, device)
    self.hidden_size = hidden_size
    
  def forward(self, sources, valid_len):
    ##############################################################################
    # TODO: Implement LSTM Encoder forward pass
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    return outputs, (h, c)

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, device):
    super(Decoder, self).__init__()
    """
    inputs:
      vocab_size: int, the number of words in the vocabulary
      embedding_dim: int, dimension of the word embedding
      hidden_size: int, dimension of vallina RNN
    """
    
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.enc = LSTM(embedding_dim+hidden_size, hidden_size, device)
    self.att = DotProductAttention()
    self.output_emb = nn.Linear(hidden_size, vocab_size)
    self.hidden_size = hidden_size
    
  def forward(self, state, target, valid_len):
    loss = 0
    preds = []
    
    ##############################################################################
    # TODO: Implement LSTM Decoder forward pass. Your solution should also use
    # self.att for attention.
    ##############################################################################
    # Replace "pass" statement with your code    
    pass
    # END OF YOUR CODE
    return loss, preds
  
  def predict(self, state, target, valid_len):
    pred = None
    ##############################################################################
    # TODO: Implement LSTM Encoder prediction. Your solution should also use
    # self.att for attention.
    ##############################################################################
    # Replace "pass" statement with your code    
    pass
    # END OF YOUR CODE
    return pred

    

In [ ]:
class NMTLSTM(nn.Module):
  def __init__(self, src_vocab_size, tgt_vocab_size, embedding_dim, hidden_size, device):
    super(NMTLSTM, self).__init__()
    self.enc = Encoder(src_vocab_size, embedding_dim, hidden_size, device)
    self.dec = Decoder(tgt_vocab_size, embedding_dim, hidden_size, device)
    
  def forward(self, src, src_len, tgt, tgt_len):
    outputs, (h, c) = self.enc(src, src_len)
    loss, pred = self.dec((outputs, (h, c), src_len), tgt, tgt_len)
    return loss, pred
  
  def predict(self, src, src_len, tgt, tgt_len):
    outputs, (h, c) = self.enc(src, src_len)
    pred = self.dec.predict((outputs, (h, c), src_len), tgt, tgt_len)
    return pred


In [ ]:
def train_lstm(net, train_iter, lr, epochs, device):
  # training
  net = net.to(device)

  optimizer = torch.optim.Adam(net.parameters(), lr=lr)
  loss_list = []
  print_interval = len(train_iter)
  total_iter = epochs * len(train_iter)
  for e in range(epochs):
    net.train()
    for i, train_data in enumerate(train_iter):
      train_data = [ds.to(device) for ds in train_data]

      loss, pred = net(*train_data)

      loss_list.append(loss.mean().detach())
      optimizer.zero_grad()
      loss.mean().backward()
      optimizer.step()

      step = i + e * len(train_iter)
      if step % print_interval == 0:
        print('iter {} / {}\tLoss:\t{:.6f}'.format(step, total_iter, loss.mean().detach()))
        print('pred:\t {}\n'.format(pred[0].detach().cpu()))
        print('tgt:\t {}\n'.format(train_data[2][0][1:].cpu()))
  return loss_list

seed(1)
batch_size = 32
lr = None
##############################################################################
# TODO: Find a good learning rate to train this model. Make sure your best
# model is saved to the `lstm_net` variable.
##############################################################################
# Replace "pass" statement with your code
pass
# END OF YOUR CODE
epochs = 50

embedding_dim = 250
hidden_size = 128

vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size)
lstm_net = NMTLSTM(vocab_eng.num_word, vocab_fra.num_word, embedding_dim, hidden_size, device)

lstm_loss_list = train_lstm(lstm_net, train_iter, lr, epochs, device)

### LSTM Loss Curve

Plot the loss curve over time.

In [ ]:
if device != "cpu":
    lstm_loss_list_cpu = [ele.cpu() for ele in lstm_loss_list]
else:
    lstm_loss_list_cpu = lstm_loss_list
plt.plot(np.arange(len(lstm_loss_list_cpu)), lstm_loss_list_cpu)
plt.title('Loss Curve of LSTM Attention')

Test the accuracy of your model. You should be able to get at least 80% accuracy.

In [ ]:
def comp_acc(pred, gt, valid_len):
  N, T_gt = gt.shape[:2]
  _, T_pr = pred.shape[:2]
  assert T_gt == T_pr, 'Prediction and target should have the same length.'
  len_mask = torch.arange(T_gt).expand(N, T_gt)
  len_mask = len_mask < valid_len[:, None]
  
  pred_crr = (pred == gt).float() * len_mask.float() # filter out the 'bos' token
  pred_acc = pred_crr.sum(dim=-1) / (valid_len - 1).float() # minus the 'bos' token
  return pred_acc
  
def evaluate_lstm(net, train_iter, device):
  acc_list = []
  for i, train_data in enumerate(train_iter):
    train_data = [ds.to(device) for ds in train_data]

    pred = net.predict(*train_data)

    pred_acc = comp_acc(pred.detach().cpu(), train_data[2].detach().cpu()[:, 1:], train_data[3].cpu())
    acc_list.append(pred_acc)
    if i < 5:# print 5 samples from 5 batches
      pred = pred[0].detach().cpu()
      pred_seq = []
      for t in range(MAX_LEN+1):
        pred_wd = vocab_fra.index2word[pred[t].item()] 
        if pred_wd != 'eos':
          pred_seq.append(pred_wd)

      print('pred:\t {}\n'.format(pred_seq))
      print('tgt:\t {}\n'.format([vocab_fra.index2word[t.item()] for t in train_data[2][0][1:].cpu()]))

  print('Prediction Acc.: {:.4f}'.format(torch.cat(acc_list).mean()))
  
seed(1)
batch_size = 32

vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size)
evaluate_lstm(lstm_net, train_iter, device)

## Transformers

Recurrent Neural Networks can capture long-range, variable-length sequential information, but updating the current state relies on the previous states. Thus it cannot be parallelized across the entire sequence. In contrast, CNNs are easy to parallelize but they cannot capture sequential dependency within variable-length sequences and their receptive field is limited. Transformers resolve this dilemna by being able to capture long-range dependencies while also being easy to parallelize.

Transformers consist of several different components and we will walk you through each of them. The original paper can be found [here](https://arxiv.org/pdf/1706.03762.pdf). [Here](http://jalammar.github.io/illustrated-transformer/) is a very informative blog about transformers that should also be a good reference.

**Multi-Head Self-Attention**

Multi-head self-attention is an extension of the dot-product attention we've previously implemented. The "self-attention" part means that the query, key, and value all come from the same sequence. For a sentence, this means that we are looking at how each word pays attention to other words in the same sentence. The "multi-head" part means instead of only having one attention map, we can have multiple. This means that for a given word in the sentence, it can pay attention to different parts of the sentence.

The steps in the multi-head attention can be summarzied by the following steps:

   1. The multi-head self-attention takes the initial query $Q$, key $K$, and value $V$ as input. Note that, if not provided specifically, usually these are set to the same input embeddings $X=Q=K=V$ initially.
   
   1. Then, a linear projection is applied to $Q,K,V$ sepearately for each head $i=1,\dots,h$. 
      $$
   Q_i = QW^{Q}_i, K_i = KW^{K}_i, V_i = VW^{V}_i, i \in [0, \dots, h-1],
   $$
   
   where $W^Q_i \in \mathcal{R}^{d_{model} \times d_k}, W^K_i \in \mathcal{R}^{d_{model} \times d_k}\text{, and } W^V_i \in \mathcal{R}^{d_{model} \times d_v}$.
     
   1. Apply the scaled dot-product attention to each of these projected set of queries, keys, and values:
   $$
   \text{head}_i = \text{Attention}(Q_i, K_i, V_i) = \text{softmax}(\frac{Q_iK^T_i}{\sqrt{d_k}})V_i
   $$
   
   1. Concatenate all the heads together and project it with another learned linear projections: 
   
   $$
   \text{O} = \text{Concate(head}_1, \dots, \text{head}_h) \\
   \text{MultiHead}(Q, K, V) = \text{O}W^o, \hspace{10mm} \text{where } W^o \in \mathcal{R}^{{hd_v} \times d_{model}}
   $$

A good visualization from the above referenced [blog](http://jalammar.github.io/illustrated-transformer/) is shown below. Transformer stacks several multi-head attention modules together. For the first multi-head layer, the input is from the dataset, so an additional embedding layer is needed to project the input sequence into the appropriate dimensions. For subsequent layers, the output from the layer previous layer is directly used as input.

<div>
    <img src="http://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png" width="600"/>
</div>

Image source: http://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png

Implement the MultiHeadAttention class below:
 - Complete the __init__() function, where the linear mappings for query, key, values, and output should be created.
 - Complete the forward() function, where the multi-head attention is performed.

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, dk, num_heads,  **kwargs):
    super(MultiHeadAttention, self).__init__()
    """
    Inputs:
      d_model: int, the same d_model in paper, feature dimension of query/key/values
      d_k: int, feature projected dimension of query/key/value, we follow the setting in the paper, where d_v=d_k=d_q
      num_heads: int, number of heads used for this MultiHeadAttention
    """
    self.num_heads = num_heads
    self.attention = DotProductAttention()
    ##############################################################################
    # TODO: Initialize the linear mappings for the query, key, and values.
    # Also initialize the weight matrix for the output.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

  def forward(self, query, key, value, valid_length):
    """
    inputs:
      query: tensor of size (B, T, d_model)
      key: tensor of size (B, T, d_model)
      value: tensor of size (B, T, d_model)
      valid_length: (B, )

      B is the batch_size, T is length of sequence, d_model is the feature dimensions of query,
      key, and value.

    Outputs:
      attention (B, T, d_model)
      """
    ##############################################################################
    # TODO: Implement the forward pass of MultiHeadAttention.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    return attention

In [ ]:
cell = MultiHeadAttention(5, 90, 9)
X = torch.ones((2, 4, 5))
valid_len = torch.tensor([2, 3])
cell(X, X, X, valid_len).shape

### Position-wise Feed-Forward Network

Another key component in the Transformer block is the position-wise feed-forward network (FFN). It's called position-wise FFN because the linear mapping is applied to each position separately and identically. For example, for an embedded input of size $N \times T \times D_{in}$, there are $N*T$ vectors of dimension $D_{in}$. If we apply a one layer position-wise FFN with weights of size $D_{in} \times D_{out}$. The linear projection will be applied to each of the $N*T$ vectors separately and identically. Thus, the output would have size $N \times T \times D_{out}$. Another way to think about this is that this is the same as a 1x1 convolution mapping from $D_{in}$ channels to $D_{out}$ channels.

Transformers stack two layers of position-wise FFN together, with a ReLU activation in between:

$$
\text{PositionWiseFFN}(x) = \text{max}(0, xW_1 + b_1)W_2 + b_2
$$

Complete the class PositionWiseFFN:

- Complete the __init__() function, where two position-wise FFN should be created.
- Complete the forward() function.

In [ ]:
class PositionWiseFFN(nn.Module):
  def __init__(self, input_size, ffn_l1_size, ffn_l2_size):
    super(PositionWiseFFN, self).__init__()
    """
    Inputs:
      input_size: int, feature dimension of the input
      fnn_l1_size: int, feature dimension of the output after the first position-wise FFN.
      fnn_l2_size: int, feature dimension of the output after the second position-wise FFN.
    """
    ##############################################################################
    # TODO: Initialize the feed forward network for PositionWiseFFN
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

  def forward(self, X):
    """
    Input:
      X: tensor of size (N, T, D_in)
    Output:
      o: tensor of size (N, T, D_out)
    """
    o = None
    ##############################################################################
    # TODO: Implement forward pass of PositionWiseFFN
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    return o



Check your result. Expected output

```
[[ 0.1609,  0.0371,  0.4916,  0.1781,  0.2010,  0.0161,  0.0869, -0.1879],
        [ 0.1609,  0.0371,  0.4916,  0.1781,  0.2010,  0.0161,  0.0869, -0.1879],
        [ 0.1609,  0.0371,  0.4916,  0.1781,  0.2010,  0.0161,  0.0869, -0.1879]]
```

In [ ]:
seed(1)
ffn = PositionWiseFFN(4, 4, 8)
ffn(torch.ones((2, 3, 4)))[0]

### Positional Encoding

Replacing RNNs with the multi-head attention layer and applying the position-wise feed-forward network makes the computation parallelizable since these modules compute the output of each item in the sequence independently. However, since every item is processed in parallel, there is no notion of ordering of the sequence. For an input sentence, this means that the transformer doesn't know the ordering of the words in the sentence. For most tasks, this ordering is very important. To address this, transformers propose adding a positional encoding to each input that corresponds to the position in the sequence. This means that we take the position of each word in the sentence (eg. 0, 1, 2, etc...) and map it to some $d_{model}$-dimensional embedding. We then add this embedding with every input item so that the input is not position-aware. Transformers use the following sinusoidal positional encoding:

$$
\begin{align*}
PE_{(pos, 2i)} &= sin(pos / 10000^{2i/d_{model}}) \\
PE_{(pos, 2i+1)} &= cos(pos / 10000^{2i/d_{model}}) 
\end{align*}
$$

An example borrowed from this [blog](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/) can give an ituition how this positional encoding works. Suppose you want to encode the number from $0$ to $8$ using binary encoding, the result would like this:
$$
\begin{align*}
0: && 0  0  0 \\
1: && 0  0  1 \\
2: && 0  1  0 \\
3: && 0  1  1 \\
4: && 1  0  0 \\
5: && 1  0  1 \\
6: && 1  1  0 \\
7: && 1  1  1 \\
\end{align*}
$$
Note the frequency of ones in each digit is different. Thus, words at different locations will have different embedding features (digits in the example). The figure below visualized a position encoding matrix of dimension $\mathcal{R}^{50 \times 128}$

The forward pass of the positional encoding should add the positional embedding to the input:

`y = x + positional_encoding(x)`

<div>
    <img src="https://d33wubrfki0l68.cloudfront.net/ef81ee3018af6ab6f23769031f8961afcdd67c68/3358f/img/transformer_architecture_positional_encoding/positional_encoding.png" width="600"/>
</div>

Image source: https://d33wubrfki0l68.cloudfront.net/ef81ee3018af6ab6f23769031f8961afcdd67c68/3358f/img/transformer_architecture_positional_encoding/positional_encoding.png

Complete the class PositionalEncoding:
- Complete the __init__() function, where the tensor $PE$ should be created.
- Complete the forward() function.

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, dim, device, max_len=1000):
    super(PositionalEncoding, self).__init__()
    """
    Inputs:
      dim: feature dimension of the positional encoding
    """
    ##############################################################################
    # TODO: Initialize positional encoding. You should create `self.pe`
    # here according to the definition above. The positional encoding should
    # support up to position `max_len`.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE


  def forward(self, X):
    """
    Inputs:
      X: tensor of size (N, T, D_in)
    Output:
      Y: tensor of the same size of X
    """
    Y = None
    ##############################################################################
    # TODO: Implement forward pass for positional encoding. After getting the positional
    # encoding with regards to the time dimension, add it to the input X.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    return Y

Check your result. Expected output

```
tensor([[1.0000, 2.0000, 1.0000, 2.0000, 1.0000, 2.0000, 1.0000, 2.0000, 1.0000,
         2.0000],
        [1.8415, 1.5403, 1.1578, 1.9875, 1.0251, 1.9997, 1.0040, 2.0000, 1.0006,
         2.0000],
        [1.9093, 0.5839, 1.3117, 1.9502, 1.0502, 1.9987, 1.0080, 2.0000, 1.0013,
         2.0000],
        [1.1411, 0.0100, 1.4578, 1.8891, 1.0753, 1.9972, 1.0119, 1.9999, 1.0019,
         2.0000],
        [0.2432, 0.3464, 1.5923, 1.8057, 1.1003, 1.9950, 1.0159, 1.9999, 1.0025,
         2.0000]])
```

In [ ]:
seed(1)
pe = PositionalEncoding(10, device)
pe(torch.ones((2, 5, 10), device=device))[0]

### Add and Norm

Transformers use a residual connection followed by a layer normalization layer to connect the inputs and outputs of other layers. To be specific, an "add and norm" layer is appended after each multi-head attention layer and the position-wise FFN layer. *The code for AddNorm Layer is given as below.*

In [ ]:
class AddNorm(nn.Module):
    def __init__(self, dropout, embedding_size):
        super(AddNorm, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(embedding_size)

    def forward(self, X, Y):
        return self.norm(self.dropout(Y) + X)

### Encoder and Decoder

The following figure gives a simple example of how the Transformer is built on these components introduced above. It's easy to see that the encoder of the Transformer consists of several identical encoder blocks, and so does the decoder.

<div>
<img src="http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png" width="600"/>
</div>

Image source: http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png

Complete the forward() function for the EncoderBlock and DecoderBlock. Note that, for the decoder, when applying self-attention, the sequential queries **cannot** attend to those at later time steps. For example, in a sequence, the query entry at time step 5 can only observe the first 5 entries. You can use `valid_length` to enforce this.


In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self, d_model, d_k, ffn_l1_size, ffn_l2_size, num_heads, dropout):
    super(EncoderBlock, self).__init__()
    """
    Inputs:
      d_model: int, feature dimension of query/key/value
      d_k: int, feature projected dimension of query/key/value, we follow the setting in the paper, where d_v=d_k=d_q
      fnn_l1_size: int, feature dimension of the output after the first position-wise FFN.
      fnn_l2_size: int, feature dimension of the output after the second position-wise FFN.
      num_heads: int, number of head for multi-head attention layer.
      dropout: dropout probability for dropout layer.
      
    """
    self.attention = MultiHeadAttention(d_model, d_k, num_heads)
    self.addnorm_1 = AddNorm(dropout, d_model)
    self.ffn = PositionWiseFFN(d_model, ffn_l1_size, ffn_l2_size)
    self.addnorm_2 = AddNorm(dropout, d_model)

  def forward(self, X, valid_length):
    """
    Inputs:
      X: tensor of size (N, T, D), embedded input sequences
      valid_length: tensor of size (N), valid lengths for each sequence
    """
    Y = None
    ##############################################################################
    # TODO: Implement forward pass for the EncoderBlock. Use the figure above
    # for guidance:
    # attention -> add+norm -> feed forward -> add+norm
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    return Y

In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self, d_model, d_k, ffn_l1_size, ffn_l2_size, num_heads,
             dropout, **kwargs):
    super(DecoderBlock, self).__init__()
    """
    Inputs:
      d_model: int, feature dimension of query/key/value
      d_k: int, feature projected dimension of query/key/value, we follow the setting in the paper, where d_v=d_k=d_q
      fnn_l1_size: int, feature dimension of the output after the first position-wise FFN.
      fnn_l2_size: int, feature dimension of the output after the second position-wise FFN.
      num_heads: int, number of head for multi-head attention layer.
      dropout: dropout probability for dropout layer.
      
    """
    self.attention_1 = MultiHeadAttention(d_model, d_k, num_heads)
    self.addnorm_1 = AddNorm(dropout, d_model)
    self.attention_2 = MultiHeadAttention(d_model, d_model, num_heads)
    self.addnorm_2 = AddNorm(dropout, d_model)
    self.ffn = PositionWiseFFN(d_model, ffn_l1_size, ffn_l2_size)
    self.addnorm_3 = AddNorm(dropout, d_model)

  def forward(self, X, **kwargs):
    """
    Inputs:
      X: tensor of size (N, T, D), embedded input sequences
      **kwargs: other arguments you think is necessary for implementation
    Outputs:
      Y: tensor of size (N, T, D_out)
      
      Feel free to output variables if necessary.
    """
    Y = None
    ##############################################################################
    # TODO: Implement forward pass for the DecoderBlock. Use the figure above
    # for guidance:
    # self attention -> add+norm -> enc-dec attention -> add+norm -> feed forward -> add+norm
    # for the first attention layer, make sure to construct a `valid_length` that
    # ensures each element cannot attend to later elements in the sequence.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    return Y

### Transformer  Implementation

By stacking two encoder blocks and two decoder blocks, build the Transformer using the above components. 

- Implement the Encoder of Transformer:
 - Complete the __init__() function with a word embedding layer and several EncoderBlocks.
 - Complete the forward() function
- Implement the Decoder of Transformer
 - Complete the __init__() function
 - Complete the forward() function
- Implement the Transformer
 - Complete the forward() function
 - Complete the predict() function


In [ ]:
class TransformerEncoder(nn.Module):
  def __init__(self, vocab_size, d_model, ffn_l1_size, ffn_l2_size,
               num_heads, num_layers, dropout, device):
    super(TransformerEncoder, self).__init__()
    """
    Inputs:
      d_model: int, feature dimension of query/key/value
      fnn_l1_size: int, feature dimension of the output after the first position-wise FFN.
      fnn_l2_size: int, feature dimension of the output after the second position-wise FFN.
      num_heads: int, number of head for multi-head attention layer.
      dropout: dropout probability for dropout layer.
      num_layers: number of encoder blocks
    """
    ##############################################################################
    # TODO: Implement init() function for TransformerEncoder. See forward() notes
    # for more details.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

  def forward(self, X, valid_length):
    ##############################################################################
    # TODO: Implement forward pass for the TransformerEncoder
    # First, use an embedding so each element in X is d_model (hint: use nn.Embedding)
    # Then, apply the positional embedding to each element
    # Lastly, pass the resulting input into num_layers of EncoderBlocks
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE

    return X

In [ ]:
class TransformerDecoder(nn.Module):
  def __init__(self, vocab_size, d_model, ffn_l1_size, ffn_l2_size,
             num_heads, num_layers, dropout, device):
    super(TransformerDecoder, self).__init__()
    """
    Inputs:
      d_model: int, feature dimension of query/key/value
      fnn_l1_size: int, feature dimension of the output after the first position-wise FFN.
      fnn_l2_size: int, feature dimension of the output after the second position-wise FFN.
      num_heads: int, number of head for multi-head attention layer.
      dropout: dropout probability for dropout layer.
      num_layers: number of decoder blocks
    """
    ##############################################################################
    # TODO: Implement init() function for TransformerDecoder
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE


  def forward(self, X, state):
    """
    Inputs:
      X: tensor of size (N, T, D), embedded input sequences
      valid_length: tensor of size (N,), valid lengths for each sequence
    """
    ##############################################################################
    # TODO: Implement forward pass for the TransformerDecoder. This will look
    # very similar to the TransformerEncoder.
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    return self.dense(X), X

In [ ]:
class Transformer(nn.Module):
  """The base class for the encoder-decoder architecture."""
  def __init__(self, encoder, decoder, **kwargs):
    super(Transformer, self).__init__(**kwargs)
    self.encoder = encoder
    self.decoder = decoder

  def forward(self, src_array, src_valid_len, tgt_array, tgt_valid_len):
    """Forward function"""
    loss = 0
    pred = None
    ##############################################################################
    # TODO: Implement forward pass of transformer
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    return loss, pred
        
  def predict(self, src_array, src_valid_len, tgt_array, tgt_valid_len):
    pred = None
    ##############################################################################
    # TODO: Implement predict() of transformer
    ##############################################################################
    # Replace "pass" statement with your code
    pass
    # END OF YOUR CODE
    return pred



Find a good learning rate for training this model. Feel free to tune other hyperparameters as well as long as your best model is saved in `transformer_net`.

In [ ]:
def train_transformer(net, train_iter, lr, epochs, device):
  # training
  net = net.to(device)

  optimizer = torch.optim.Adam(net.parameters(), lr=lr)
  loss_list = []
  print_interval = len(train_iter)
  total_iter = epochs * len(train_iter)
  for e in range(epochs):
    net.train()
    for i, train_data in enumerate(train_iter):
      train_data = [ds.to(device) for ds in train_data]

      loss, pred = net(*train_data)

      loss_list.append(loss.mean().detach())
      optimizer.zero_grad()
      loss.mean().backward()
      optimizer.step()

      step = i + e * len(train_iter)
      if step % print_interval == 0:
        print('iter {} / {}\tLoss:\t{:.6f}'.format(step, total_iter, loss.mean().detach()))
        print('pred:\t {}\n'.format(pred[0].detach().cpu()))
        print('tgt:\t {}\n'.format(train_data[2][0][1:].cpu()))
  return loss_list


# hyper-params: feel free to modify the values and numbers of hyper-params 

# training
seed(1)
batch_size = 32
vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size)

embedding_dim = 250
hidden_size = 128

#transformer hp
# d_model = 512 # for GPU training
d_model = 64 # for CPU training
# ffn_l1_size = 2048 # for GPU training
ffn_l1_size = 128 # for CPU training
ffn_l2_size = d_model
num_heads = 8
num_layers = 6
dropout = 0.1

lr = None
##############################################################################
# TODO: Find a good learning rate to train this model. Make sure your best
# model is saved to the `transformer_net` variable. Feel free to tune other hyperparameters
# as well.
##############################################################################
# Replace "pass" statement with your code
pass
# END OF YOUR CODE
epochs = 50

encoder = TransformerEncoder(vocab_eng.num_word, d_model, ffn_l1_size, ffn_l2_size,
                             num_heads, num_layers, dropout, device=device)
decoder = TransformerDecoder(vocab_fra.num_word, d_model, ffn_l1_size, ffn_l2_size,
                             num_heads, num_layers, dropout, device=device)
transformer_net = Transformer(encoder, decoder)

transformer_loss_list = train_transformer(transformer_net, train_iter, lr, epochs, device)

In [ ]:
if device != "cpu":
    transformer_loss_list_cpu = [ele.cpu() for ele in transformer_loss_list]
else:
    transformer_loss_list_cpu = transformer_loss_list
plt.plot(np.arange(len(transformer_loss_list_cpu)), transformer_loss_list_cpu)
plt.title('Loss Curve of Transformer')

Test the accuracy of your model. You should be able to get at least 90% accuracy if you use GPU (with a larger hidden dimension) and at least 45% if you use CPU (with a smaller hidden dimension). 

In [ ]:
def comp_acc(pred, gt, valid_len):
  N, T_gt = gt.shape[:2]
  _, T_pr = pred.shape[:2]
  assert T_gt == T_pr, 'Prediction and target should have the same length.'
  len_mask = torch.arange(T_gt).expand(N, T_gt)
  len_mask = len_mask < valid_len[:, None]
  
  pred_crr = (pred == gt).float() * len_mask.float() # filter out the 'bos' token
  pred_acc = pred_crr.sum(dim=-1) / (valid_len - 1).float() # minus the 'bos' token
  return pred_acc
  
def evaluate_transformer(net, train_iter, device):
  net.eval()
  acc_list = []
  for i, train_data in enumerate(train_iter):
    train_data = [ds.to(device) for ds in train_data]

    pred = net.predict(*train_data)

    pred_acc = comp_acc(pred.detach().cpu(), train_data[2].detach().cpu()[:, 1:], train_data[3].cpu())
    acc_list.append(pred_acc)
    if i < 5:# print 5 samples from 5 batches
      pred = pred[0].detach().cpu()
      pred_seq = []
      for t in range(MAX_LEN+1):
        pred_wd = vocab_fra.index2word[pred[t].item()] 
        if pred_wd != 'eos':
          pred_seq.append(pred_wd)

      print('pred:\t {}\n'.format(pred_seq))
      print('tgt:\t {}\n'.format([vocab_fra.index2word[t.item()] for t in train_data[2][0][1:].cpu()]))

  print('Prediction Acc.: {:.4f}'.format(torch.cat(acc_list).mean()))
  
seed(1)
batch_size = 32

vocab_eng, vocab_fra, train_iter = load_data_nmt(batch_size)

evaluate_transformer(transformer_net, train_iter, device)